In [18]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp

In [19]:
import time 
import re
import os
import trp
import boto3
import minecart
import json
import logging 

import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session

In [20]:
# initiate s3 bucket and corresponding data folder
bucket = "ran-s3-systemic-risk"
data_folder ="Input/X-17A-5-Subsets/"

# script to perform OCR (using Textract) for X-17A-5 subsets
out_folder = 'Output/X-17A-5-BS/'

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# AWS Asynchronous Textract Script (requesting Job)
**Content modified from Amazon AWS Textract repository (refer to [URL](https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py) below)** 

In [21]:
def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job for table analysis 
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3BucketName,
                'Name': objectName
            }
        },
        FeatureTypes=['FORMS', 'TABLES']    # selecting forms and tables from the OCR
    )
    
    # return response job ID for service
    return response["JobId"]

In [22]:
def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(1)
    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_analysis(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    
    return status

In [23]:
def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):
        response = client.get_document_analysis(JobId=jobId, NextToken=nextToken)
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [24]:
def runJob(bucket:str, key:str):
    """
    Function designed to call an AWS Textract 
    """
    # S3 storage for files on AWS site   
    jobId = startJob(bucket, key)   # intialize Textract job 
    print("Started job with id: {}".format(jobId))

    # if job is complete from AWS return response object 
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
        
    return response

# AWS Extraction Scripts (Key-Value Pairs)
**The content was modified from AWS to extract key-value pairs in form documents from Block objects that are stored in a map. (refer to [URL](https://docs.aws.amazon.com/textract/latest/dg/examples-extract-kvp.html))**

In [25]:
def get_kv_relationship(key_map, value_map, block_map):
    """
    Retreaving the Key-Value relationship from FORM 
    """
    key_value_map = {}
    
    # unpack the key_map to retrieve the block id and key names
    for block_id, key_block in key_map.items():
        # retrieve value block given job id
        value_block = find_value_block(key_block, value_map)

        # get text value from key and value blocks
        key = get_text(key_block, block_map)
        val = get_text(value_block, block_map)
        
        # map the key and value pairs
        key_value_map[key] = val
        
    return key_value_map

In [26]:
def find_value_block(key_block, value_map):
    """
    Retrieving value block from AWS textract job 
    """
    for relationship in key_block['Relationships']:
        if relationship['Type'] == 'VALUE':
            for value_id in relationship['Ids']:
                value_block = value_map[value_id]
    return value_block

In [27]:
def get_text(result, blocks_map):
    """
    Retrieving text values from given work
    """
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    
                    # if word then we append to space
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] == 'SELECTED':
                            text += 'X '    
    return text

# OCR Wrapper Functions
**The scripts perform an OCR job from AWS Textract, and returning well formated data**

In [28]:
def trp2df(table:trp.Table) -> pd.DataFrame:
    """
    Function designed to convert a trp table into a dataframe
    :param table: a trp table object parsed from a pdf  
    :return: a DataFrame object housing a textracted trp table
    
    Complexity -> O(n^2) approx.
    """
    N = len(table.rows)               # number of rows in table
    M = len(table.rows[0].cells)      # number of columns in table
    arr = [0]*N
    
    # iterate through each row within the provided table
    for row in np.arange(N):
        
        # strip the text from the cell references to construct (N X M) matrix
        arr[row] = [table.rows[row].cells[col].text.strip() for col in np.arange(M)]
        
    return pd.DataFrame(arr)

In [29]:
def readTable(response:list) -> pd.DataFrame:
    """
    Function to transform AWS Textract object to a dataframe, by searching for tables
    :param response: AWS Textract response object
    """
    # in the event multiple tables detected on one page (concat them)
    catDF = []
    
    # format the Textract response type 
    doc = trp.Document(response)
    
    # iterate through document pages
    for page in doc.pages:
        
        # itterate through page tables
        for table in page.tables: 
            
            # convert trp-table into dataframe object
            df = trp2df(table)
            
            # remove columns that are completly empty
            empty_cols = [col for col in df.columns if (df[col] == '').all()]
            df = df.drop(empty_cols, axis=1)
  
            # number of columns in dataframe
            n = df.columns.size
            
            # reset the column names (avoid the column names)
            df.columns = np.arange(n)
            
            ##############################################################
            #                           NOTES
            #          a good dataframe should have 2-3 columns
            #      anything more or less is a reading error we ignore
            ##############################################################
            
            # if the dataframe has more than 3 columns then we most likley have an issue in parsing
            if n > 3:
                return None
            
            elif n > 1:
                
                ##############################
                # Balance Sheet Assummptions
                ##############################
                
                lineIndex = df.columns[0]

                # check for the word "cash" in a string at the begining, ignoring case sensitivity (asset check)
                assetCheck = df[lineIndex].str.contains('^Cash', regex=True, flags=re.IGNORECASE)

                # check for the word "Liabilities" in a string at the end, ignoring case sensitivity (liability check)
                debtCheck1 = df[lineIndex].str.contains('Liabilities$|^Liabilities', regex=True, flags=re.IGNORECASE)
                debtCheck2 = df[lineIndex].str.contains('Liability$|^Liability', regex=True, flags=re.IGNORECASE)
                
                # check for the presence of $ sign, we assume the balance sheet items should have presence of $ sign
                dollarCheck = df[df.columns[1]].str.contains('\$[^\]]+', regex=True, flags=re.IGNORECASE)
                
                ##############################
                # Balance Sheet Determination
                ##############################
                
                # check if the key words have been found 
                check1 = df[assetCheck | debtCheck1 | debtCheck2].empty      # check for terms, and $ presence
                check2 = df[dollarCheck == True].empty                       # check for presence of '$' sign  
                check3 = df[debtCheck1 == True].empty                        # debt check for Liabilities
                check4 = df[debtCheck2 == True].empty                        # debt check for Liability 
                
                # if either asset term or liability term is found, with a $ sign we append the dataframe
                if not check1 and not check2:
                    catDF.append(df)      # we append since sometimes asset and liablility tables are seperated 

                    if not check3 or not check4:
                        # if liability table was found on the first iteration we simply concat data frames and return 
                        return pd.concat(catDF)
        

In [40]:
def readForm(response:list) -> dict:
    """
    Function to transform AWS Textract object to a dictionary, by searching for key value pairs
    :param response: AWS Textract response object
    """
    # format the Textract response type 
    doc = trp.Document(response)
    
    # initializing 
    key_map = {}
    value_map = {}
    block_map = {}

    # iterate through document pages
    for page in doc.pages:

        # itterate through page tables
        for block in page.blocks: 

            # store the block id in map to retrive information later
            block_id = block['Id']
            block_map[block_id] = block

            # if Key-value set has been seen 
            if block['BlockType'] == "KEY_VALUE_SET":

                # if KEY is labeled as entity type then we found Key, else we found VALUE
                if 'KEY' in block['EntityTypes']:
                    key_map[block_id] = block
                else:
                    value_map[block_id] = block

    return get_kv_relationship(key_map, value_map, block_map)

## Extract Balance Sheet information

In [44]:
def textractParse(path:str, index:int, csvDirectory:np.ndarray, bucket:str = "ran-s3-systemic-risk", 
                  out_folder:str = 'Output/X-17A-5-BS/') -> dict:
    """
    Function runs a Textract job and saves Balance Sheet information to .csv file in s3 folder 
    """
    errors = ''
    
    # track the presence of json file storing information on forms
    if os.path.exists('X17A5-forms.json'):
        with open('X17A5-forms.json', 'r') as f: forms = json.loads(f.read())
    else:
        forms = {}
    
    # baseFile name to name export .csv file e.g. 782124-2002.csv
    baseFile = '-'.join(path.split('/')[-1].split('-')[:2])
    fileName = baseFile + '.csv'
    print('\nPerforming OCR for {}'.format(baseFile))

    # if file is not found in directory we continue the iteration process
    if (out_folder + fileName not in csvDirectory) or (baseFile not in forms):

        # temporary data frame object for balance sheet information
        res = runJob("ran-s3-systemic-risk", path)
        
        # if Textract job did not fail we continue extraction
        if res[0]['JobStatus'] != 'FAILED':
            # retrieve structured data pulls from OCR
            forms[baseFile] = readForm(res)
            tempDF = readTable(res)
            
            print(tempDF)
            
            # checks for type of return, if none then we log an error
            if type(tempDF) == pd.DataFrame:
                
                # writing data frame to .csv file
                tempDF.to_csv(fileName, index=False)

                # save contents to AWS S3 bucket
                with open(fileName, 'rb') as data:
                    s3.put_object(Bucket=bucket, Key=out_folder + fileName, Body=data)

                # remove local file after it has been created
                os.remove(fileName)
                
                print('-----------------------------------------------------')
                print('Saved {} file to s3 bucket'.format(baseFile + '.csv'))
            else:
                errors = 'No Balance Sheet found, or parsing error'
        else:
            errors = 'Could not parse, JOB FAILEDs'
    else:
        print('{} has been downloaded'.format(fileName))
        
    # storing key-value line items
    with open('X17A5-forms.json', 'w') as f: json.dump(forms, f)
        
    return errors

In [45]:
# csv Directory to store balance sheet information 
csvs = np.array(session.list_s3_files(bucket, out_folder))

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))[1:]

errorDict = {}

# iterate through X-17A-5 subsets stored in s3 
for i, key in enumerate(paths):     
    val = textractParse(key, i, csvs)
    
    if val != '':
        errorDict[key] = val


Performing OCR for 1224385-2004
1224385-2004.csv has been downloaded

Performing OCR for 1224385-2005
1224385-2005.csv has been downloaded

Performing OCR for 1224385-2006
1224385-2006.csv has been downloaded

Performing OCR for 1224385-2007
1224385-2007.csv has been downloaded

Performing OCR for 1224385-2008
1224385-2008.csv has been downloaded

Performing OCR for 1224385-2009
1224385-2009.csv has been downloaded

Performing OCR for 1224385-2010
1224385-2010.csv has been downloaded

Performing OCR for 1224385-2011
1224385-2011.csv has been downloaded

Performing OCR for 1224385-2012
1224385-2012.csv has been downloaded

Performing OCR for 1224385-2013
1224385-2013.csv has been downloaded

Performing OCR for 1224385-2014
Started job with id: 25002a39bac325b04a35264a8d3db8560a70faf1195e041dd5d4e54bbd4cae6b
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
                                 

Started job with id: 7dfd36665fcf941b11a7c1a421dea45850ef4ab4083cd3fbffd6fc468b431e46
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
                                                    0              1
0                                                Cash      $ 170,575
1     Cash segregated pursuant to federal regulations        569,989
2   Financial instruments owned, at fair value ($2...     45,293,482
3                                 Securities borrowed     18,532,966
4     Securities purchased under agreements to resell     22,305,354
5   Receivable from broker-dealers and clearing or...     21,632,676
6                           Receivable from customers      4,093,015
7                              

Resultset page recieved: 6
Resultset page recieved: 7
                                                    0              1
0                                                Cash      $ 111,424
1   Cash segregated under federal and other regula...      2,361,878
2                                 Securities borrowed     31,926,833
3     Securities purchased under agreements to resell     21,350,116
4   Receivable from broker-dealers and clearing or...     18,717,937
5                      Receivable from customers, net      7,125,995
6   Financial instruments owned, at fair value ($4...     59,637,269
7   Property, equipment and leasehold improvements...          6,261
8                                            Goodwill         79,687
9                          Receivable from affiliates         21,102
10                                       Other assets        496,040
11                                       Total assets  $ 141,834,542
12                    Liabilities and Member's Eq

Started job with id: b1b04ee1e0281ad42588330f17eaead6e27f04ca621ba1976625331ecb67ce69
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
                                                    0              1
0                           Cash and cash equivalents    $ 4,604,370
1   Cash and securities segregated in compliance w...               
2                               and other regulations     21,969,289
3   Receivables from brokers, dealers and clearing...      7,976,880
4       Receivables from customers and counterparties     10,289,117
5                                 Securities borrowed    151,340,243
6     Securities purchased under agreements to resel

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_P

...

Started job with id: ae5d6d5b02149d7e568fa9da60d97ae27088cf6988d3c9f1aea00ac2769a73bc
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
Resultset page recieved: 8
                                                    0         1          2
0                                              Assets                     
1                           Cash and cash equivalents                $ 730
2   Cash segregated and on deposit for Federal and...                     
3            or deposited with clearing organizations               14,085
4     Collate

Started job with id: b0a70320c21d344b43715b3e2c3bfd78b547ffd3052966e0461a313bc2d7e8e2
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
Resultset page recieved: 8
                                                    0         1           2
0                                              Assets                      
1                           Cash and cash equivalents                 $ 699
2   Cash segregated and on deposit for Federal and...                10,691
3     Collateralized short-term financing agreements:                      
4   Secu

Started job with id: a24f62ee13113f4f768837f39339e593eb778dd2eba24385fc83bfc33140d7fb
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
None

Performing OCR for 91154-2016
Started job with id: 62a0d70020985b8b0ed731c6dcfd550e09a9b81d21c403d3fa5197e53486514e
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job s

Started job with id: 5eb4d11d3a5286d27d9e2d3ff17cf5464ecd85b3d2dd162d7c8485eb5f3aed31
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
Resultset page recieved: 8
                                                    0          1
0                                              Assets           
1                           Cash and cash equivalents      $ 471
2   Cash segregated under federal and other regul

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
Resultset page recieved: 8
                                                    0          1
0                                              Assets           
1                                                Cash      $ 798
2   Cash segregated under federal and other regula...      6,055
3   Securities borrowed and purchased under agreem...    171,131
4   Trading account assets ($26,266 pledged as col...           
5                          Mortgage-backed securities     29,560
6         U.S. Treasury and federal agency securities     18,175
7                                   Equity securities     12,389
8                           Corporate debt securities    

In [46]:
# storing unique list of asset items and liability line items
with open('textractErrors.json', 'w') as f: json.dump(errorDict, f)